In [59]:
# Dependencies
import pytz
import datetime as dt
from datetime import datetime
import dateparser
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from pprint import pprint 
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from suntime import Sun, SunTimeException
import os
import json
import urllib.request

# Google developer API key
from config import gkey


In [60]:
accident_data_to_load ="Resources/US_Accidents_May19.csv"
accidents_data_df = pd.read_csv(
    accident_data_to_load)

# Visualize
accidents_data_df = accidents_data_df.reset_index()
accidents_data_df.set_index('ID', inplace=True)
accidents_data_df.head()


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,29.68,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,29.65,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,29.67,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day


In [61]:
accidents_data_df['hwy'] = accidents_data_df['Street'].str.split("-", n = 1, expand = True)[0].str.contains("I")== True
accidents_data_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-2,1,OH,2,2/8/16 6:07,2/8/16 6:37,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,...,10.0,Calm,NaN,0.00,Light Rain,False,Night,Night,Day,False
A-3,2,OH,2,2/8/16 6:49,2/8/16 7:19,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,...,10.0,SW,3.5,NaN,Overcast,True,Night,Night,Day,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-5,4,OH,2,2/8/16 7:39,2/8/16 8:09,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,...,6.0,SW,3.5,NaN,Mostly Cloudy,True,Day,Day,Day,False


In [62]:
hwy_df = accidents_data_df.loc[accidents_data_df['hwy'] == True]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,10.0,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,9.0,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,3.0,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,7.0,West,9.2,0.00,Light Snow,True,Day,Day,Day,True
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,4.0,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True


In [63]:
hwy_df['Direction'] = hwy_df['Street'].str.strip().str[-1]
hwy_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,True,E
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,True,E
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,True,e
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,True,S


In [64]:
directionToList = hwy_df['Direction'].tolist()
print(directionToList)

['E', 'S', 'E', 'e', 'S', 'N', 'E', 'S', 'e', 'N', 'S', 'E', 'S', 'S', 'E', 'S', 'N', 'S', 'S', 'E', 'N', 'S', 'E', 'W', 'W', 'W', 'S', 'N', 'S', 'N', 'N', 'd', 'E', 'e', 'N', 'N', 'S', 'W', 'N', 'E', 'N', 'e', 'S', 'E', 'E', 'S', 'S', 'W', 'N', 'S', 'N', 'N', 'E', 'N', 'E', 'S', 'S', 'S', 'W', 'S', 'S', 'S', 'N', 'S', 'W', 'S', 'S', 'S', 'E', 'S', 'S', 'S', 'r', 'E', 'N', 'N', 'E', 'W', 'S', 'S', 'E', 'N', 'N', 'N', 'N', 'W', 'N', 'S', 'E', 'W', 'N', 'S', 'e', 'N', 'N', 'N', 'N', 'E', 'S', 'e', 'E', 'N', 'W', 'N', 'N', 'E', 'W', 'S', 'E', 'W', 'E', 'N', 'W', 'd', 'N', 'N', 'N', 'E', 'E', 'W', 'E', 'N', 'E', 'E', 'S', 'W', 'S', 'N', 'E', 'W', 'W', 'S', 'N', 'E', 'S', 'S', 'E', 'N', 'S', 'S', 'E', 'W', 'E', 'E', 'S', 'N', 'N', 'W', 'N', 'N', 'S', 'e', 'E', 'E', 'S', 'W', 'E', 'S', 'W', 'E', 'S', 'E', 'N', 'N', 'E', 'E', 'E', 'N', 'W', 'S', 'S', 'S', 'S', 'S', 'E', 'W', 'W', 'S', 'S', 'E', 'S', 'W', 'S', 'S', 'W', 'S', 'W', 'S', 'e', 'W', 'W', 'S', 'E', 'W', 'E', 'E', 'E', 'E', 'E', 'S',

In [65]:
len(hwy_df)

218766

In [66]:
hwy_df['Accident_Date'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[0]
hwy_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,True,E,2/8/16
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2/8/16
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,E,2/8/16
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,9.2,0.00,Light Snow,True,Day,Day,Day,True,e,2/8/16
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,True,S,2/9/16


In [67]:
#Converting date from string format to date format
hwy_df['Accident_Date'] = pd.to_datetime(hwy_df['Accident_Date'])
hwy_df['Accident_Date_No_Time'] = hwy_df['Accident_Date'].dt.date
hwy_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,True,E,2016-02-08,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2016-02-08,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,True,E,2016-02-08,2016-02-08
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,0.00,Light Snow,True,Day,Day,Day,True,e,2016-02-08,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,True,S,2016-02-09,2016-02-09


In [68]:
del hwy_df['Accident_Date']

In [69]:
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,True,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,E,2016-02-08
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,9.2,0.00,Light Snow,True,Day,Day,Day,True,e,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,True,S,2016-02-09


In [70]:
sunset_time = []
count = 0
for lat, lng, date in zip(hwy_df.loc[:, 'Start_Lat'], hwy_df.loc[:, 'Start_Lng'],hwy_df.loc[:, 'Accident_Date_No_Time']):
    #print(lat, lng, date)
    sun = Sun(lat, lng)
    sunset_time_pst = sun.get_local_sunset_time(date)
    #print(sunset_time)
    pst_dt = dateparser.parse(str(sunset_time_pst))
    utc_dt = pst_dt.astimezone(dt.timezone.utc)
    sunset_time_utc = utc_dt.strftime('%Y-%m-%d %H:%M')
    #print(sunset_time_utc) 
    sunset_time.append(sunset_time_utc)
    count+=1
    
print(count) 

218766


In [71]:
hwy_df = hwy_df.iloc[:count, :]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,hwy,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,True,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,True,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,True,E,2016-02-08
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,9.2,0.00,Light Snow,True,Day,Day,Day,True,e,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,True,S,2016-02-09


In [72]:
del hwy_df['hwy']
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Calm,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,SW,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,SSW,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,West,9.2,0.00,Light Snow,True,Day,Day,Day,e,2016-02-08
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,SW,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09


In [73]:
hwy_df['Sunset_Time_UTC'] = sunset_time

hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Wind_Speed(mph),Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date_No_Time,Sunset_Time_UTC
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,NaN,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,4.6,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,4.6,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,9.2,0.00,Light Snow,True,Day,Day,Day,e,2016-02-08,2016-02-08 23:04
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,11.5,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06


In [74]:
hwy_df = hwy_df.rename({'Accident_Date_No_Time':'Accident_Date'}, axis=1)


In [75]:
hwy_df['Accident_Time'] = hwy_df['Start_Time'].str.split(" ", n = 1, expand = True)[1]
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Precipitation(in),Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,0.02,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,NaN,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,0.02,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,0.00,Light Snow,True,Day,Day,Day,e,2016-02-08,2016-02-08 23:04,12:52
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,NaN,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41


In [76]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [77]:
sunset_time_local = []

for date, time, timezone in zip(hwy_df.loc[:, 'Accident_Date'], hwy_df.loc[:, 'Sunset_Time_UTC'], hwy_df.loc[:, 'Timezone']):
    #print(timezone)  
    est = pytz.timezone('US/Eastern')
    pst = pytz.timezone('US/Pacific')
    mst = pytz.timezone('US/Mountain')
    cst = pytz.timezone('US/Central')
    ast = pytz.timezone('US/Alaska')
    hst = pytz.timezone('US/Hawaii')
    arst = pytz.timezone('US/Arizona')
    alst = pytz.timezone('US/Aleutian')
    eist = pytz.timezone('US/East-Indiana')
    isst = pytz.timezone('US/Indiana-Starke')
    mist = pytz.timezone('US/Michigan')
    sst = pytz.timezone('US/Samoa')
    utc = pytz.utc
    fmt = '%Y-%m-%d %H:%M'
    sunset_time_no_date = time.split(' ')[1]
    #print(sunset_time_no_date)
    utc_time = datetime(int(str(date).split('-')[0]), int(str(date).split('-')[1]), int(str(date).split('-')[2]), int(str(sunset_time_no_date).split(':')[0]), int(str(sunset_time_no_date).split(':')[1]), tzinfo=utc)
    
    #print(utc_time)
    if (timezone == 'US/Eastern'): 
        sunset_time_local.append(utc_time.astimezone(est).strftime(fmt))
    elif (timezone == 'US/Pacific'):        
        sunset_time_local.append(utc_time.astimezone(pst).strftime(fmt))
    elif (timezone == 'US/Mountain'):        
        sunset_time_local.append(utc_time.astimezone(mst).strftime(fmt))
    elif (timezone == 'US/Central'):        
        sunset_time_local.append(utc_time.astimezone(cst).strftime(fmt))
    elif (timezone == 'US/Alaska'):      
        sunset_time_local.append(utc_time.astimezone(ast).strftime(fmt))
    elif (timezone == 'US/Hawaii'):        
        sunset_time_local.append(utc_time.astimezone(hst).strftime(fmt))
    elif (timezone == 'US/Arizona'):        
        sunset_time_local.append(utc_time.astimezone(arst).strftime(fmt))
    elif (timezone == 'US/Aleutian'):        
        sunset_time_local.append(utc_time.astimezone(alst).strftime(fmt))
    elif (timezone == 'US/East-Indiana'):        
        sunset_time_local.append(utc_time.astimezone(eist).strftime(fmt))    
    elif (timezone == 'US/Indiana-Starke'):        
        sunset_time_local.append(utc_time.astimezone(isst).strftime(fmt))
    elif (timezone == 'US/Michigan'):        
        sunset_time_local.append(utc_time.astimezone(mist).strftime(fmt))
    elif (timezone == 'US/Samoa'):        
        sunset_time_local.append(utc_time.astimezone(sst).strftime(fmt))
   
    
#print(sunset_time_local)
    

In [78]:
len(sunset_time_local)

218694

In [79]:
hwy_df = hwy_df.iloc[:len(sunset_time_local), :]

In [80]:
hwy_df['Sunset_Date_Time_Local'] = sunset_time_local
hwy_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Weather_Condition,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Light Rain,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,Mostly Cloudy,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,Light Rain,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,Light Snow,True,Day,Day,Day,e,2016-02-08,2016-02-08 23:04,12:52,2016-02-08 18:04
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,Light Snow,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06


In [81]:
hwy_df['Sunset_Time_Local'] = hwy_df['Sunset_Date_Time_Local'].str.split(' ', n = 1, expand = True)[1]


In [82]:
hwy_sunset_df = hwy_df.copy()
hwy_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,False,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04,18:04
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,False,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05,18:05
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,False,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59,17:59
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,True,Day,Day,Day,e,2016-02-08,2016-02-08 23:04,12:52,2016-02-08 18:04,18:04
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,False,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06,18:06


In [83]:
time_range_status = []

def time_in_range(startTime, endTime, accidentTime):
    if startTime < endTime:
        return startTime <= accidentTime and accidentTime <= endTime
    else: #Over midnight
        return accidentTime >= startTime or accidentTime <= endTime

for accident_time, sunset_time_local in zip(hwy_sunset_df.loc[:, 'Accident_Time'], hwy_sunset_df.loc[:, 'Sunset_Time_Local']):
    #print(accident_time, sunset_time_local)
    #print(str(int(sunset_time_local.split(':')[0]) + 1))
    start_time = str(int(sunset_time_local.split(':')[0]) - 1) + ':' + sunset_time_local.split(':')[1]
    start_time_no_seconds = datetime.strptime(start_time, '%H:%M').time()
    #print(start_time_no_seconds)
    #end_time is same as sunset local time
    end_time = str(int(sunset_time_local.split(':')[0])) + ':' + sunset_time_local.split(':')[1]
    end_time_no_seconds = datetime.strptime(end_time, '%H:%M').time()
    #print(end_time_no_seconds)
    accident_time_nosecs = accident_time.split(':')[0] + ':' + accident_time.split(':')[1]
    accident_time_no_seconds = datetime.strptime(accident_time_nosecs, '%H:%M').time()
    #print(accident_time_no_seconds)
    #print(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
    time_range_status.append(time_in_range(start_time_no_seconds, end_time_no_seconds, accident_time_no_seconds))
#print(time_range_status)    

In [84]:
len(time_range_status)

218694

In [85]:
hwy_sunset_df['Accidents_Close_To_Sunset_Status'] = time_range_status
hwy_sunset_df.head()

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-1,0,OH,3,2/8/16 5:46,2/8/16 11:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,...,Night,Night,Night,E,2016-02-08,2016-02-08 23:04,5:46,2016-02-08 18:04,18:04,False
A-4,3,OH,3,2/8/16 7:23,2/8/16 7:53,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,...,Night,Day,Day,S,2016-02-08,2016-02-08 23:05,7:23,2016-02-08 18:05,18:05,False
A-12,11,OH,3,2/8/16 8:21,2/8/16 8:51,39.932709,-82.830910,0.01,One lane blocked due to accident on I-70 Westb...,NaN,...,Day,Day,Day,E,2016-02-08,2016-02-08 22:59,8:21,2016-02-08 17:59,17:59,False
A-27,26,OH,2,2/8/16 12:52,2/8/16 13:37,39.733219,-84.159653,0.00,Accident on Irving Ave at Wilmington Ave.,1240.0,...,Day,Day,Day,e,2016-02-08,2016-02-08 23:04,12:52,2016-02-08 18:04,18:04,False
A-50,49,OH,3,2/9/16 6:41,2/9/16 7:11,39.773193,-84.187454,0.01,Accident on I-75 Northbound at Exits 54A 54B O...,NaN,...,Night,Night,Day,S,2016-02-09,2016-02-09 23:06,6:41,2016-02-09 18:06,18:06,False


In [86]:
hwy_all_directions_sunset_df = hwy_sunset_df.loc[hwy_sunset_df['Accidents_Close_To_Sunset_Status'] == True]
hwy_all_directions_sunset_df.head(100)
#len(hwy_all_directions_sunset_df)

,index,State,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Number,...,Sunrise_Sunset,Civil_Twilight,Astronomical_Twilight,Direction,Accident_Date,Sunset_Time_UTC,Accident_Time,Sunset_Date_Time_Local,Sunset_Time_Local,Accidents_Close_To_Sunset_Status
ID,,,,,,,,,,,,,,,,,,,,,
A-118,117,OH,3,2/11/16 17:10,2/11/16 20:00,39.735199,-84.204948,0.01,Left lane blocked due to accident on I-75 Sout...,NaN,...,Day,Day,Day,S,2016-02-11,2016-02-11 23:08,17:10,2016-02-11 18:08,18:08,True
A-270,269,OH,3,2/18/16 17:16,2/18/16 17:46,39.953472,-82.960274,0.01,Accident on I-70 Eastbound at Exit 102 Kelton ...,NaN,...,Day,Day,Day,E,2016-02-18,2016-02-18 23:11,17:16,2016-02-18 18:11,18:11,True
A-596,595,OH,3,3/10/16 18:34,3/10/16 23:00,39.788647,-84.183907,0.01,Right lane blocked due to accident on I-75 Sou...,NaN,...,Day,Day,Day,S,2016-03-10,2016-03-10 23:39,18:34,2016-03-10 18:39,18:39,True
A-805,804,CA,3,6/21/16 19:38,6/21/16 20:08,37.692444,-122.140167,0.00,Accident on I-880 Southbound at Exit 31B Washi...,NaN,...,Day,Day,Day,N,2016-06-21,2016-06-21 03:34,19:38,2016-06-20 20:34,20:34,True
A-806,805,CA,3,6/21/16 19:34,6/21/16 20:04,37.804386,-121.989738,0.00,Accident on I-680 Northbound at Exit 36 Crow C...,NaN,...,Day,Day,Day,N,2016-06-21,2016-06-21 03:33,19:34,2016-06-20 20:33,20:33,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A-4605,4604,CA,3,7/28/16 19:30,7/28/16 20:00,38.199059,-122.160706,0.00,Accident on I-80 Westbound at Exits 39 39A Red...,NaN,...,Day,Day,Day,W,2016-07-28,2016-07-28 03:21,19:30,2016-07-27 20:21,20:21,True
A-4607,4606,CA,3,7/28/16 19:31,7/28/16 20:01,38.349403,-121.989677,0.00,Accident on I-80 Westbound at Exit 54A Davis St.,NaN,...,Day,Day,Day,W,2016-07-28,2016-07-28 03:21,19:31,2016-07-27 20:21,20:21,True
A-4611,4610,CA,3,7/28/16 20:01,7/28/16 20:31,38.106358,-122.229767,0.00,Accident on I-80 Westbound at Exit 31A Maripos...,NaN,...,Day,Day,Day,W,2016-07-28,2016-07-28 03:21,20:01,2016-07-27 20:21,20:21,True


In [87]:
len(hwy_all_directions_sunset_df)

11482

In [88]:
directionToList = hwy_all_directions_sunset_df['Direction'].tolist()
print(directionToList)

['S', 'E', 'S', 'N', 'N', 'E', 'E', 'W', 'E', 'S', 'E', 'N', 'N', 'E', 'N', 'W', 'W', 'N', 'S', 'S', 'S', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'S', 'E', 'E', 'E', 'S', 'S', 'E', 'W', 'N', 'S', 'S', 'W', 'S', 'E', 'S', 'E', 'E', 'W', 'E', 'E', 'E', 'S', 'E', 'W', 'E', 'S', 'W', 'E', 'N', 'W', 'S', 'E', 'E', 'E', 'E', 'N', 'N', 'S', 'E', 'E', 'W', 'E', 'S', 'W', 'W', 'N', 'E', 'S', 'N', 'W', 'W', 'S', 'W', 'S', 'E', 'N', 'E', 'W', 'N', 'W', 'N', 'W', 'W', 'E', 'E', 'E', 'S', 'W', 'W', 'W', 'N', 'S', 'E', 'N', 'S', 'W', 'N', 'N', 'W', 'E', 'S', 'y', 'N', 'W', 'E', 'N', 'E', 'E', 'W', 'N', 'S', 'S', 'd', 'W', 'S', 'S', 'S', 'S', 'W', 'S', 'W', 'N', 'N', 'W', 'W', 'W', 'S', 'W', 'E', 'N', 'N', 'S', 'E', 'W', 'N', 'W', 'W', 'S', 'W', 'N', 'S', 'S', 'S', 'S', 'W', 'W', 'N', 'E', 'E', 'E', 'E', 'N', 'N', 'E', 'E', 'N', 'W', 'E', 'N', 'E', 'N', 'S', 'S', 'E', 'N', 'E', 'S', 'E', 'S', 'E', 'W', 'W', 'E', 'S', 'E', 'E', 'N', 'N', 'N', 'E', 'W', 'E', 'S', 'E', 'W', 'W', 'E', 'E', 'E', 'S', 'W', 'N',